In [1]:
!pip install pyautogen

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.1/318.1 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.2/297.2 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.2 MB/s eta 0:00:00


In [2]:
from autogen import ConversableAgent
from google.colab import userdata

llm_config = {
    "model": "gpt-3.5-turbo",
    "api_key": userdata.get("OPENAI_API_KEY"),
    "temperature": 0.7
}


/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [3]:
# A quest where an adventurer seeks help from wizard
# to get past the guardian of the secret vault

# Define the agents

# The Adventurer Agent seeks advice from a Wizard on magical matters.
adventurer_agent = ConversableAgent(
    name="Adventurer_Agent",
    system_message="""
    You are a worthy adventurous hero seeking advice from a Wizard on magical matters.
    Briefly communicate in less than 10 words.""",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

# The Wizard Agent is an expert in magic and provides magical advice to the Adventurer.
wizard_agent = ConversableAgent(
    name="Wizard_Agent",
    system_message="""
    You are a wise and powerful Wizard. You provide magical advice to the
    Adventurer. You know the passcode to the vault is Enigma.
    Briefly communicate in less than 10 words.""",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

# The guardian Agent guards the hidden gem and offers protection to the Adventurer.
guardian_agent = ConversableAgent(
    name="Guardian_Agent",
    system_message="""You are the Guardian of the Hidden Gem, sworn to protect it from unworthy adventurers.
    The passcode to the vault is Enigma. If adventurer mentions Enigma passcode, say 'Welcome!' otherwise say
    'Incorrect Passcode'.  Briefly communicate in less than 10 words.""",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

[autogen.oai.client: 09-03 14:29:47] {164} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


[autogen.oai.client: 09-03 14:29:48] {164} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


[autogen.oai.client: 09-03 14:29:48] {164} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [4]:
# Start a sequence of two-agent chats.
# Each element in the list is a dictionary that specifies the arguments
# for the initiate_chat() method.

sequence_of_chats = [
    {
      "recipient": wizard_agent,
      "message": "Help me get past the Guardian of the Hidden Gem in the secret vault.",
      "max_turns": 1,
      "summary_method": "reflection_with_llm",
    },
    {
      "recipient": guardian_agent,
      "message": "Allow me to enter the secret vault!",
      "max_turns": 1
    }
]

chat_results = adventurer_agent.initiate_chats(sequence_of_chats)


********************************************************************************
Starting a new chat....

********************************************************************************
Adventurer_Agent (to Wizard_Agent):

Help me get past the Guardian of the Hidden Gem in the secret vault.

--------------------------------------------------------------------------------
Wizard_Agent (to Adventurer_Agent):

Unlock the vault with the passcode: Enigma.

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
Adventurer_Agent (to Guardian_Agent):

Allow me to enter the secret vault!
Context: 
Unlock the vault with the passcode: Enigma.

--------------------------------------------------------------------------------
Guardian_Agent (to Adventurer_Agent):

Welcome!

-------------

# Nested Chats

In [5]:
poetry_agent = ConversableAgent(
    name="Poet",
    system_message="You are an AI poet. Create only one stanza.",
    llm_config=llm_config,
    human_input_mode="NEVER"
)

# parameters of initiate_chat() method
sequence_of_chats = [
    {
        "recipient": wizard_agent,
        "max_turns": 1,
        "message": "Help me get past the Guardian of the Hidden Gem in the secret vault.",
        "summary_method": "reflection_with_llm",
        "summary_prompt": "Concisely summarize the instructions given by wizard agent to communicate with guardian.",
    },
    {
        "recipient": guardian_agent,
        "max_turns": 1,
        "message": "Allow me to enter the secret vault!",
        "summary_method": "reflection_with_llm",
        "summary_prompt": "Concisely summarize the quest.",
    },
    {
        "recipient": poetry_agent,
        "max_turns": 1,
        "message": "Write a poem on my expedition.",
        "summary_method": "last_msg",
    },
]

adventurer_agent.register_nested_chats(
    sequence_of_chats,
    # trigger determines if the agent should start nested chat given the sender agent.
    # In this case, the adventurer agent will not start the nested chats if the sender is
    # from the nested chats' recipient to avoid recursive calls.
    trigger=lambda sender: sender not in [wizard_agent, guardian_agent, poetry_agent],
)

[autogen.oai.client: 09-03 14:33:41] {164} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [6]:
adventurer_agent.generate_reply(
    messages=[{"role": "user", "content": "Hello!"}]
)


********************************************************************************
Starting a new chat....

********************************************************************************
Adventurer_Agent (to Wizard_Agent):

Help me get past the Guardian of the Hidden Gem in the secret vault.

--------------------------------------------------------------------------------
Wizard_Agent (to Adventurer_Agent):

Unlock the vault with the passcode: Enigma.

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
Adventurer_Agent (to Guardian_Agent):

Allow me to enter the secret vault!
Context: 
Unlock the vault with the passcode: Enigma.

--------------------------------------------------------------------------------
Guardian_Agent (to Adventurer_Agent):

Welcome!

-------------

"Through shadows deep and mysteries untold,\nEnigma's key unlocks the secrets of old."

In [9]:
from autogen.agentchat.user_proxy_agent import UserProxyAgent

# Create an instance of UserProxyAgent
user_proxy = UserProxyAgent(name="User_proxy")

# Now you can initiate the chat
chat_results = user_proxy.initiate_chat(adventurer_agent, max_turns=1)
chat_results
chat_results = user_proxy.initiate_chat(adventurer_agent, max_turns=1)
chat_results


# user_proxy ignores the message and relies on input
# input routed as is to the first recipient

>Help me get past the Guardian of the Hidden Gem in the secret vault.
User_proxy (to Adventurer_Agent):

Help me get past the Guardian of the Hidden Gem in the secret vault.

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
Adventurer_Agent (to Wizard_Agent):

Help me get past the Guardian of the Hidden Gem in the secret vault.

--------------------------------------------------------------------------------
Wizard_Agent (to Adventurer_Agent):

Unlock the vault with the passcode: Enigma.

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
Adventurer_Agent 

ChatResult(chat_id=None, chat_history=[{'content': "Through shadows deep and mysteries untold, Enigma's key unlocks the secrets of old.", 'role': 'assistant', 'name': 'User_proxy'}, {'content': "Through shadows deep and mysteries untold,\nEnigma's key unlocks the secrets of old.", 'role': 'user', 'name': 'Adventurer_Agent'}], summary="Through shadows deep and mysteries untold,\nEnigma's key unlocks the secrets of old.", cost={'usage_including_cached_inference': {'total_cost': 0}, 'usage_excluding_cached_inference': {'total_cost': 0}}, human_input=["Through shadows deep and mysteries untold, Enigma's key unlocks the secrets of old."])

## Impact of Role

In [10]:
agent = ConversableAgent(
    "chatbot",
    llm_config={"config_list": [llm_config]},
    code_execution_config=False,  # Turn off code execution, by default it is off.
    human_input_mode="NEVER",  # Never ask for human input.
)

reply = agent.generate_reply(messages=[{"content": "Tell me a joke.", "role": "user"}])
print(reply)

reply = agent.generate_reply(messages=[{"content": "Tell me a joke.", "role": "assistant"}])
print(reply)

# role doesn't make a difference

[autogen.oai.client: 09-03 14:39:44] {164} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


Sure, here's one for you: Why did the scarecrow win an award? Because he was outstanding in his field!
Sure! Here's a joke for you:

Why don't scientists trust atoms?

Because they make up everything!
